In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import plotting_utils
from plotting_utils import *
import altair as alt
import altair_saver
alt.data_transformers.disable_max_rows()

In [ ]:
alt.renderers.enable('default')

In [ ]:
alt.themes.enable('personal')
jobs = [
# #     load_jobs('pv100_v4*', subdir='exploration'),
#     load_jobs('pv100_clustercheck_v1*', subdir='exploration'),
#     load_jobs('pv100_clustercheck_v1*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement'),
    
# #     load_jobs('pv100_clustercheck_v2*', subdir='exploration'),
# #     load_jobs('pv100_clustercheck_v2*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement'),

#     load_jobs('pv100_clustercheck_v3*', subdir='exploration'),
#     load_jobs('pv100_clustercheck_v3*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement'),

    load_jobs('pv100_bbe_v3*', subdir='exploration', title='SAC + UFO'),
    load_jobs('pv100_bbe_v3*', subdir='bbe', title='SAC + BBE'),
    
#     load_jobs('pv100_bbe_v4*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement', title='SAC + UFO'),
#     load_jobs('pv100_bbe_v4*', subdir='bbe', root='/home/will/vlg5/exploration-reimplement', title='SAC + BBE'),
    
    load_jobs('pv100_sacqex_v2*', subdir='sac_qex', title='SAC Qex'),
]
data = pd.concat(jobs, sort=False)

# for cluster in ['zaan', 'cassio', 'greene']:
#     data.loc[data.name.str.contains(cluster), 'cluster'] = cluster

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="PV100", 
    color='title',
    strokeDash='policy_updates_per_step:O',
    detail='policy_updates_per_step:O',
    window=10,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'elapsed', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [
    load_jobs('hallway_v2*', subdir='exploration'),
#     load_jobs('hallway_v2*velocity_1*no_explorationTrue', subdir='exploration', title="SAC velocity 1"),
#     load_jobs('hallway_v2*velocity_4*no_explorationTrue', subdir='exploration', title="SAC velocity 4"),
#     load_jobs('hallway_v2*no_explorationFalse', subdir='exploration'),
#     load_jobs('hallway_v3*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Hallway", 
    opacity='use_exploration',
    color='task',
#     color='name',
    window=0,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('hallway_distractor_v1*no_explorationTrue*', subdir='exploration', title="SAC"),
#     load_jobs('hallway_distractor_v1*no_explorationFalse*', subdir='exploration'),
#     load_jobs('hallway_distractor_v2*', subdir='exploration'),
#     load_jobs('hallway_distractor_v3_policyupdates*', subdir='exploration'),
#     load_jobs('hallway_distractor_v5*', subdir='exploration'),
#     load_jobs('hallway_distractor_v7*', subdir='exploration'),
    
    load_jobs('hallway_distractor_v8*no_explorationTrue*', subdir='exploration', title="SAC, 1 update"),
#     load_jobs('hallway_distractor_v8*no_explorationFalse*discount0.97*', subdir='exploration', title="1 update, novelty discount 0.97"),
    load_jobs('hallway_distractor_v8*no_explorationFalse*discount0.99*', subdir='exploration', title="1 update, novelty discount 0.99"),
    load_jobs('hallway_distractor_v9*no_explorationTrue*', subdir='exploration', title="SAC, 2 updates"),
#     load_jobs('hallway_distractor_v9*no_explorationFalse*discount0.97*', subdir='exploration', title="2 updates, novelty discount 0.97"),
    load_jobs('hallway_distractor_v9*no_explorationFalse*discount0.99*', subdir='exploration', title="2 updates, novelty discount 0.99"),
    load_jobs('hallway_distractor_v10*no_explorationTrue*', subdir='exploration', title="SAC, 4 updates"),
# #     load_jobs('hallway_distractor_v10*no_explorationFalse*discount0.97*', subdir='exploration', title="4 u pdates, novelty discount 0.97"),
    load_jobs('hallway_distractor_v10*no_explorationFalse*discount0.99*', subdir='exploration', title="4 updates, novelty discount 0.99"),
]
data = pd.concat(jobs, sort=False)

N = 100
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset,
    title="Hallway distractor", 
    opacity='policy_updates_per_step',
    color='use_exploration',
#     color='title',
#     color='policy_updates_per_step:O',
#     opacity='use_exploration',z
    window=5,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [
    load_jobs('hallway_inverse_distractor_v2*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 100
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset,
    title="Hallway inverse distractor", 
#     opacity='policy_updates_per_step',
    color='task',
#     color='title',
#     color='policy_updates_per_step:O',
#     opacity='use_exploration',z
    window=0,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('pv100_v5*', subdir='exploration'),
#     load_jobs('pv100_v6*', subdir='exploration'),
#     load_jobs('pv100_v8*', subdir='exploration'),
    load_jobs('pv100_kernel_guidance_v1*', subdir='exploration'),
    load_jobs('pv100_kernel_guidance_v2*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 200
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Point velocity", 
    color='density_state_scale:O',
    strokeDash='n_updates_per_step',
    detail='n_updates_per_step',
    window=10,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'density_size', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

# chart = make_base_chart(
#     subset[subset.test], 
#     title="Manipulator reach-lift", 
#     color='use_exploration',
#     opacity='density_state_scale:O',
#     window=10,
#     y='smoothed_score:Q',
#     tooltip=['density_state_scale']
# )
# chart.chart.interactive()

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('wex_walk_narrow_sparse_v1*', subdir='exploration'),
    
#     load_jobs('wex_walk_narrow_sparse_v1*scale0.3*', subdir='exploration'),
#     load_jobs('wex_walk_narrow_sparse_v2*', subdir='exploration'),
    load_jobs('wex_walk_narrow_sparse_v3*', subdir='exploration'),
    load_jobs('walker_walk_v1*', subdir='exploration'),
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
]
data = pd.concat(jobs, sort=False)
data = data[(data.env == 'walker') | (data.env == 'walker_explore')]
data['exploration'] = 'Undirected'
data.loc[data.title == 'SAC + BBE', 'exploration'] = 'BBE'
data.loc[data.title == 'SAC + BBE', 'use_exploration'] = False
data.loc[data.use_exploration, 'exploration'] = 'UFO'


N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Walker walk", 
    color='exploration',
    strokeDash='task',
    detail='task',
    window=5,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'elapsed', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('fex_hard_v1*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
#     load_sac_results('finger', 'turn_hard', title='SAC old'),
#     load_jobs('finger_v2*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement'),    
#     load_jobs('fex_hard_v3*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement'),
    
#     load_jobs('finger_all_v1*', subdir='exploration'), # density_state_scale 0.34, density_tolerance 0.95, 4 seeds
#     load_jobs('finger_all_v2*', subdir='exploration'), # seeds that died in v1
#     load_jobs('finger_all_v3*', subdir='exploration'), # density_state_scale 0.1, density_tolerance 0.5, 8 seeds
#     load_jobs('finger_all_v4*', subdir='exploration',  # density_state_scale 0.34, density_tolerance 0.95, 4 more seeds
#               root='/home/will/vlg5/exploration-reimplement'),
    load_jobs('finger_all_v6*', subdir='exploration'), # density_state_scale 0.1, density_tolerance 0.5, 8 seeds
#     load_jobs('finger_all_v5*', subdir='exploration',  # density_state_scale 0.34, density_tolerance 0.95, 4 more seeds
#               root='/home/will/vlg5/exploration-reimplement'),
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
]
data = pd.concat(jobs, sort=False)
data = data[(data.env == 'finger') | (data.env == 'finger_explore')]
data['exploration'] = 'Undirected'
data.loc[data.title == 'SAC + BBE', 'exploration'] = 'BBE'
data.loc[data.title == 'SAC + BBE', 'use_exploration'] = False
data.loc[data.use_exploration, 'exploration'] = 'UFO'

# data['title'] = data.title.str.replace('finger_all_v2_rerun', 'finger_all_v1')
# data['title'] = data.title.str.slice(stop=13) + data.use_exploration.astype(str)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Finger", 
    color='exploration',
    strokeDash='task',
#     opacity='density_state_scale:O',
    detail='task',
#     shape='task',
    window=20,
)

charts = []
for test, feature in [(False, 'rolling_mean_score'), (True, 'smoothed_score'), (False, 'sum_novelty'), (False, 'count_score'), ]:
    feature_chart = alt.hconcat(*[plot_with_bars(chart, feature, test=test, strip=False, title_flag=task).transform_filter(alt.datum.task == task)
              for task in subset.task.unique()])
    charts.append(feature_chart)
alt.vconcat(*charts)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'elapsed', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('reacher_explore_v3*', subdir='exploration'),
#     load_jobs('reacher_explore_v4_seeds*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement'),
#     load_jobs('reacher_explore_v5_seeds_fixed*', subdir='exploration'),
#     load_jobs('reacher_explore_v8*', subdir='exploration'), # more seeds, queued on greene
#     load_jobs('reacher_explore_v9*', subdir='exploration'), # re-running seeds that died on greene
    
#     load_jobs('reacher_v10*', subdir='exploration'), # good experiment with lots of seeds
    
    load_jobs('reacher_v10*no_explorationTrue*', subdir='exploration', title='SAC'),
#     load_jobs('ufo_all_v1*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement', title='SAC + UFO'),
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
]
data = pd.concat(jobs, sort=False)
data = data[(data.env == 'reacher') | (data.env == 'reacher_explore')]
# data['exploration'] = 'Undirected'
# data.loc[data.title == 'SAC + BBE', 'exploration'] = 'BBE'
# data.loc[data.title == 'SAC + BBE', 'use_exploration'] = False
# data.loc[data.use_exploration, 'exploration'] = 'UFO'

N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Reacher Explore", 
    color='title',
    strokeDash='task',
    detail='task',
    window=20,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'elapsed', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

# chart = make_base_chart(
#     subset, 
#     title="Reacher Explore", 
#     color='title',
#     strokeDash='task',
#     detail='task',
#     window=20,
# )

# chart.chart.mark_line().encode(y='mean(rolling_mean_score):Q').transform_filter(alt.datum.test == True).transform_filter(alt.datum.task == 'hard_fixed_init')

In [ ]:
alt.themes.enable('personal')
jobs = [
    load_jobs('bice_v6*', subdir='exploration'),
    load_jobs('bbe_all_v2*', subdir='bbe', title='SAC + BBE'),
]
data = pd.concat(jobs, sort=False)
data = data[(data.env == 'ball_in_cup') | (data.env == 'ball_in_cup_explore')]
data['exploration'] = 'Undirected'
data.loc[data.title == 'SAC + BBE', 'exploration'] = 'BBE'
data.loc[data.title == 'SAC + BBE', 'use_exploration'] = False
data.loc[data.use_exploration, 'exploration'] = 'UFO'

N = 300
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Ball-in-Cup", 
    color='exploration',
    strokeDash='env',
    detail='env',
    window=10,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'elapsed', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [    
    load_jobs('hallway_all_v4*', subdir='exploration', root='/home/will/vlg5/exploration-reimplement'),
]
data = pd.concat(jobs, sort=False)

N = 500
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Hallway", 
    color='task',
    strokeDash='use_exploration',
    detail='use_exploration',
    window=30,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'elapsed', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))



In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('pv100_sac_2x_no_explorationTrue', subdir='exploration', title='SAC'),

# #     load_jobs('arxiv2_pv100_seed*', subdir='exploration', title='arxiv results'),
#     load_jobs('arxiv2_pv100_seed*', subdir='exploration', title='arxiv UFO'),
#     load_jobs('arxiv2_pv100_noopt_seed*', subdir='exploration', title='arxiv UF'),
#     load_jobs('arxiv2_pv100_slow_seed*', subdir='slow', title='arxiv U'),
#     load_jobs('arxiv2_pv100_intrinsic_seed*', subdir='intrinsic', title='arxiv BBE'),
#     load_jobs('arxiv2_pv100_noexplore_seed*', subdir='exploration', title='arxiv undirected'),
# #     load_jobs('epv100kcdiag_sscale0.01_ascale1*', subdir='exploration', title='q policy small kernel'),
# #     load_jobs('epv100kcdiag_sscale0.1_ascale1*', subdir='exploration', title='q policy large kernel'),
#     load_jobs('pv100_sac', subdir='exploration', title='sacxploration seed 1'),
#     load_jobs('pv100_sac_seed2', subdir='exploration', title='sacxploration seed 2'),
    
#     load_jobs('pv100_sac_sscale0.01', subdir='exploration', title='sacxploration small kernel'),
#     load_jobs('pv100_sac_2x_no_exploration*', subdir='exploration'),
#     load_jobs('pv100_sac_2x_uniform*', subdir='exploration'),
#     load_jobs('pv100_sac_2x_warm500*', subdir='exploration'),
    

#     load_jobs('pv100_sac_2x_warm500_uniform_update_candidatesTrue', subdir='exploration', title='SAC + exploration'),
    
#     load_jobs('pv100_check_tol*', subdir='exploration'),
#     load_jobs('pv100_updates*', subdir='exploration'),
#     load_jobs('pv100_500_updates_n*', subdir='exploration'),

#     load_jobs('pv100_check_seed*', subdir='exploration'),
#     load_jobs('pv100_keops_v3*', subdir='exploration'),
    
    load_jobs('pv100_v4*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Point Velocity", 
    color='policy_updates_per_step:O',
    opacity='use_exploration',
#     color='title',
    window=10,
)

# plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False)
(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))
# (plot_with_bars(chart, 'policy_entropy', test=False))

## Failed or unhelpful experiments

### `walker_explore stand`: rewards are too dense, no gap

In [ ]:
alt.themes.enable('personal')
jobs = [
    load_jobs('wex_v1*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
#     load_jobs('wex_stand_narrow_sparse_v1*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 300
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Walker stand, narrow init", 
    color='density_state_scale:O',
    strokeDash='use_exploration',
    detail='use_exploration',
    window=5,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'policy_entropy', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

### `manipulator_explore`: turned out not to be solvable reliably by SAC (Q-learning diverges)

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('manipulator_explore_v2*', subdir='exploration'),
#     load_jobs('manipulator_explore_v8*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
#     load_jobs('manipulator_explore_v11*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),

    load_jobs('mex_fixed_v2*reach_lift*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Manipulator reach-lift, fixed", 
    color='density_state_scale:O',
    strokeDash='use_exploration',
    window=20,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

# chart = make_base_chart(
#     subset[subset.test], 
#     title="Manipulator reach-lift", 
#     color='use_exploration',
#     opacity='density_state_scale:O',
#     window=10,
#     y='smoothed_score:Q',
#     tooltip=['density_state_scale']
# )
# chart.chart.interactive()

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('manipulator_rslb_v1*', subdir='exploration'),
    load_jobs('mex_fixed_v2*reach_shaped_lift*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
    
    load_jobs('mex_fixed_v5*no_explorationTrue*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement', title='SAC'),
    
#     load_jobs('mex_fixed_v2*reach_shaped_lift*no_explorationTrue*state_scale1.0*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
#     load_jobs('mex_fixed_v3_tempcheck*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Manipulator shaped reach-lift, fixed", 
    color='density_state_scale:O',
    strokeDash='use_exploration',
    detail='title',
    window=10,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'density_size', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('manipulator_rslb_v1*', subdir='exploration'),
#     load_jobs('mex_fixed_v2*reach_shaped_lift*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
    
#     load_jobs('mex_fixed_v2*reach_shaped_lift*no_explorationTrue*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
#     load_jobs('mex_fixed_v3_tempcheck*', subdir='exploration'),

#     load_jobs('mex_fixed_v5*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
    
    load_jobs('mex_fixed_v5*no_explorationTrue*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
    load_jobs('mex_fixed_v6*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
    load_jobs('mex_fixed_v7_conserve*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Manipulator shaped reach-lift, fixed reset", 
    color='temperature:O',
    detail='title',
    strokeDash='use_exploration',
    window=10,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'density_size', test=False))
# (plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

In [ ]:
alt.themes.enable('personal')
jobs = [
    load_jobs('manipulator_v5*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
#     load_jobs('manipulator_keops_v3*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
#     load_jobs('manipulator_keops_v4*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
]
data = pd.concat(jobs, sort=False)

N = 10000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="manipulator bring_ball", 
    color='title',
    window=20,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

### `hopper`: using exploration gives a bit of a speedup, but muddied by suboptimality of learned policies

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_sac_results('hopper', 'hop'),
    load_jobs('hopper_sac_noexplore*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement', title='SAC'),

#     load_jobs('hopper_sac_2x_density_state_scale1*', subdir='exploration'),
#     load_jobs('hopper_sac2x_uniform*', subdir='exploration'),
#     load_jobs('hopper_sac_max8192_seed2*', subdir='exploration'),
#     load_jobs('hopper_sac_max8192_density_state_scale1*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),

#     load_jobs('hopper_sizes*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
#     load_jobs('hopper_speedy_sizes*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
    
    
    load_jobs('hopper_keops*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
    load_jobs('hopper_keops*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | (((subset.episode - 3) % 5 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Hopper Hop", 
    color='title',
    window=20,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'elapsed', test=False) | plot_with_bars(chart, 'density_size', test=False))

### `acrobot`: surprisingly hard, and has no exploration. no penalty from using UFO though

In [ ]:
alt.themes.enable('personal')
jobs = [
    load_sac_results('acrobot', 'swingup'),   
    load_jobs('acrobot_swingup', subdir='exploration', root='/home/will/vlg4/exploration-reimplement',
              title='acrobot swingup, kernel scale 0.3'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Acrobot Swingup", 
    color='title',
    window=30,
)


(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False)) &\
(plot_with_bars(chart, 'elapsed', test=False).interactive() | plot_with_bars(chart, 'density_size', test=False).interactive())

### `cartpole swingup_sparse`: UFO learns a little bit faster

In [ ]:
alt.themes.enable('personal')
jobs = [
#     load_jobs('ecpkcdiagfix_sscale*', subdir='exploration'),
#     load_jobs('ecpkcdiagfix_plr*', subdir='exploration'),
#     load_jobs('ecpkcdiagfix_plr1e-4_sscale0.1_ascale1*', subdir='exploration'),
    load_jobs('ecp_tab_20x2*', subdir='exploration', title='tabular count'),
#     load_jobs('cp_noexplore*', subdir='exploration', title='no exploration'),
    load_sac_results('cartpole', 'swingup_sparse'),   
    load_jobs('ecp_kcweight_plr1e-4_sscale0.1*', subdir='exploration'),


    load_jobs('cart_temp_policy_temperature0.1*', subdir='exploration'),

    load_jobs('cart_sac_2x*', subdir='exploration'),
    
#     load_jobs('cart_approx*', subdir='exploration'),
#     load_jobs('cart_temp*', subdir='exploration'),
#     load_jobs('cart_sac', subdir='exploration'),

]
data = pd.concat(jobs, sort=False)

N = 300
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Cartpole Swingup (sparse)", 
    color='title',
    window=10,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
(plot_with_bars(chart, 'policy_entropy', test=False) | plot_with_bars(chart, 'policy_entropy', test=True))

### gridworld and pv100 with kernels

In [ ]:
alt.themes.enable('personal')
jobs = [
    load_jobs('gw20_tab*', subdir='exploration', title='tabular count'),
    load_jobs('gw20_sscale*', subdir='exploration', title='kernel count (tiny)'),
    load_jobs('gw20fixedkernel_sscale0.001*', subdir='exploration', title='kernel count (tiny, fixed)'),
    load_jobs('gw20fixedkernel_sscale0.1*', subdir='exploration', title='kernel count (medium, fixed)'),
]
data = pd.concat(jobs, sort=False)
N = 100
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Gridworld 20x20", 
    color='title',
    window=5,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False).interactive() | plot_with_bars(chart, 'count_score', test=False)) & \
plot_with_bars(chart, 'policy_entropy', test=True)

In [ ]:
alt.themes.enable('personal')
jobs = [
    load_jobs('arxiv2_pv100_seed*', subdir='exploration', title='tabular_density'),
#     load_jobs('explore_pv100_kcount*', subdir='exploration'),
#     load_jobs('explore_pv100_k*clippednum*', subdir='exploration'),
#     load_jobs('epv100kc_scale*', subdir='exploration'),
#     load_jobs('epv100kc_sscale*', subdir='exploration'),
#     load_jobs('epv100kcfix_sscale*', subdir='exploration'),
#     load_jobs('epv100kcrefix_sscale*', subdir='exploration'),
    load_jobs('epv100kcdiag_sscale*', subdir='exploration'),
]
data = pd.concat(jobs, sort=False)
N = 200
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title="Point Velocity", 
    color='title',
    window=5,
)

(plot_with_bars(chart, 'rolling_mean_score', test=False) | plot_with_bars(chart, 'smoothed_score', test=True)) & \
(plot_with_bars(chart, 'sum_novelty', test=False) | plot_with_bars(chart, 'count_score', test=False)) & \
plot_with_bars(chart, 'policy_entropy', test=True)

### SAC results from Denis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def plot(df, base=None, key='episode_reward'):
    envs = np.sort(df.env.unique())
    ncol = 3
    assert envs.shape[0] % ncol == 0
    nrow = envs.shape[0] // ncol
    fig, axs = plt.subplots(nrow, ncol, figsize=(4 * ncol, 3 * nrow))
    #runs = sorted([str(x) for x in df.run.unique()])
    for idx, env in enumerate(envs):
        data = df[df['env'] == env]
        row = idx // ncol
        col = idx % ncol
        ax = axs[row, col]
        hue_order = np.sort(data.Agent.unique())
        #import ipdb; ipdb.set_trace()
        sns.lineplot(x='step', y=key, data=data, ci='sd', hue='Agent', hue_order=hue_order, ax=ax)
        #sns.lineplot(x=‘step’, y=key, data=data, ci=‘sd’, hue=‘run_id’, ax=ax, palette=sns.color_palette(‘Blues’)[3:4])
        if base is not None:
            if env in base:
                val = base[env]['d4pg']
                ax.axhline(val, ls='--', color='black', label='D4PG ($10^8$ steps)')
                ax.legend()
        ax.set_title(f'{env}')
        #ax.set_xlim(0, 10000000)
        #ax.set_ylim(0.0, 1050.0)
        #ax.legend_.remove()
    plt.tight_layout()
    plt.show()
plot(pd.read_csv('results/sac.csv'))

In [ ]:
alt.themes.enable('personal')
env = 'reacher'
task = 'hard'
jobs = [
    load_sac_results(env, task),   
#     load_jobs('acrobot_sac*', subdir='exploration', root='/home/will/vlg4/exploration-reimplement'),
]
data = pd.concat(jobs, sort=False)

N = 1000
subset = data
# subset = subset[((subset.episode % 5 == 0) & (subset.test)) | ((subset.episode % 3 == 0) & (~subset.test))]
subset = subset[(subset['episode'] <= N)]
chart = make_base_chart(
    subset, 
    title=f"{env} {task}", 
    color='title',
    window=3,
)

plot_with_bars(chart, 'rolling_mean_score', test=True)